In [1]:
# addprocs(7)  # parallel computing

In [2]:
include("prepare_data_Sioux_simp.jl");
include("extract_data.jl");
include("tap_MSA_Sioux_simp.jl");
include("demands_adjustment_Sioux_simp_gradi.jl");

In [3]:
fcoeffs = [1, 0, 0, 0, .15];

In [4]:
tapFlowVecDict = Dict{Int64, Array{Float64, 1}}();
tapFlowDicDict = Dict{Int64, Dict{(Int64,Int64),Float64}}();

In [5]:
# get observed flow vector (corresponding to ground truth demands)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [6]:
# get initial flow vector (corresponding to initial demands)
tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], fcoeffs);

In [7]:
demandsDict[0]

Dict{(Int64,Int64),Float64} with 144 entries:
  (8,10)  => 1600.0
  (9,5)   => 800.0
  (3,6)   => 300.0
  (10,5)  => 1000.0
  (6,9)   => 400.0
  (12,4)  => 600.0
  (4,4)   => 0.0
  (9,4)   => 700.0
  (10,11) => 4000.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (11,1)  => 500.0
  (8,11)  => 800.0
  (3,1)   => 100.0
  (1,10)  => 1300.0
  (1,12)  => 200.0
  (12,7)  => 700.0
  (4,5)   => 500.0
  (9,9)   => 0.0
  (11,8)  => 800.0
  (12,8)  => 600.0
  (2,4)   => 200.0
  (6,5)   => 200.0
  (4,9)   => 700.0
  (8,4)   => 700.0
  ⋮       => ⋮

In [8]:
jacobDict = Dict{Int64, Array{Float64}}()

jacobDict[1] = jacobian(tapFlowVecDict[1], fcoeffs, capacity, free_flow_time, numLinks, numODpairs, 
numRoutes, linkRoute, odPairRoute)

132x30 Array{Float64,2}:
 -0.0   0.0   0.0  0.0   0.0  -0.0  0.0  …   1.0  -0.0  0.0   0.0  0.0   1.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0  -1.0   0.0  0.0   0.0   0.0  0.0      1.0   0.0  0.0   1.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   1.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0  …   1.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0  -0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0  -0.0  0.0   0.0
 -1.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   1.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
 -1.0   0.0   0.0  0.0   0.0   0.0  0.0  …   0.0   0.0  0.0   0.0  0.0   1.0
  0.0   0.0   1.0  0.0   0.0   0.0  0.0      0.0   0.0  0.0   0.0  0.0   0.0
  0.0   0.0   0.0  0.0  -0.0   0.0  0.0     -0.0   

In [9]:
jacobDict

Dict{Int64,Array{Float64,N}} with 1 entry:
  1 => [-0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 -0.0 0.0 …

In [10]:
descDirecDict = Dict{Int64, Array{Float64,1}}()
descDirecDict[1] = descDirec(tapFlowVecDict[1], tapFlowVecDict[0], jacobDict[1], numODpairs, numLinks)

132-element Array{Float64,1}:
 -161.126 
   57.947 
 -651.939 
  129.346 
  -85.5632
 -120.101 
   -0.0   
  -58.4533
 -389.406 
   -0.0   
  147.909 
 -312.0   
   -0.0   
    ⋮     
 -450.156 
   -0.0   
 -516.756 
   -0.0   
 -403.412 
   42.4263
   41.5019
  216.375 
 -245.098 
  460.396 
 -501.903 
 -527.534 

In [11]:
demandsVecDict = Dict{Int64, Array{Float64,1}}()

demandsVecDict[1] = demandsDicToVec(demandsDict[1])

132-element Array{Float64,1}:
   94.6641
   98.6753
  503.991 
  188.239 
  278.751 
  529.941 
  832.222 
  525.316 
 1279.29  
  485.112 
  219.564 
  108.672 
   97.0662
    ⋮     
 1425.21  
  214.147 
   97.0584
  180.058 
  600.553 
  193.828 
  186.604 
  711.807 
  638.069 
  559.622 
 2024.67  
 1346.81  

In [12]:
searchDirecDict = Dict{Int64, Array{Float64,1}}()
searchDirecDict[1] = searchDirec(demandsVecDict[1], descDirecDict[1], 20)

132-element Array{Float64,1}:
 -161.126 
   57.947 
 -651.939 
  129.346 
  -85.5632
 -120.101 
   -0.0   
  -58.4533
 -389.406 
   -0.0   
  147.909 
 -312.0   
   -0.0   
    ⋮     
 -450.156 
   -0.0   
 -516.756 
   -0.0   
 -403.412 
   42.4263
   41.5019
  216.375 
 -245.098 
  460.396 
 -501.903 
 -527.534 

In [13]:
thetaMaxDict = Dict{Int64, Float64}()
thetaMaxDict[1] = thetaMax(demandsVecDict[1], searchDirecDict[1])

0.12470290230841002

In [14]:
# # update

# demandsVecNew = similar(demandsVec)
# for i = 1:length(demandsVec)
#     demandsVecNew[i] = demandsVec[i] + (alpha_max / 81) * r_[i] 
# end